**texto en negrita**![texto del vínculo](https://drive.google.com/file/d/1ll7E7Os7i7Fhi_RwW7g08dnowna9uW4j/view?usp=drive_link)

PROYECTO : CLASIFICACION USANDO NAIVE BAYES

INTEGRANTES DEL GRUPO:
1. XXXXXX , CARNET No.
2.
3. **texto en negrita**


1. IMPORTAR LIBRERIAS

In [72]:
# 1.1 ) Cargar la herramiento joblib que servira para guardar y descargar el modelo entrenado en Google Drive de tal manera que tengamos el modelo
 #disponible sin necesidad de correr todo el programa de la nueve del Colaboratory
!pip install joblib
 # Cargar la API de Kaggle para leer el dataset desde la web
!pip install kaggle

In [73]:
# 1.2 ) Importar las librerías estándar de análisis
# Libreria pandas servira crear los data frame asi como manipular y analizar datos
import pandas as pd
# Libreria numpy servira para manejar vectores y matrices de ser necesarios asi como matematica de alto nivel
import numpy as np
#from collections import defaultdict, esta libreria nos permitira crear diccionarios predetermiandos
from collections import defaultdict
# Permite cargar el  modelo entrenado de machine learning Naive Bayes a Google Drive
from joblib import dump

import random, itertools
import re
import string
import os
import sys



2. IMPORTAR DATOS

In [74]:
# 2.1) Se define la ruta para leer los archivos a usar desde la nube Google Colaboratory
     # a Google Drive
BASE_FOLDER = '/content/drive/My Drive/Colab Notebooks/PROYECTO IA/'

# Instruccion que permite importar y montar una unidad de Drive de Google drive en Google colaboratory
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
# 2.2) Ruta para leer el archivo en Google Drive
file_path = BASE_FOLDER+ "naivebayes.py"

# Verificamos si existe el archivo en Google Drive
import os
if os.path.exists(file_path):
    print("✅ Archivo encontrado en Google Drive")
else:
    print("❌ Archivo NO encontrado. Revisar la ruta.")

✅ Archivo encontrado en Google Drive


In [76]:
# 2.3) Importar la clase Naive Bayes desde Google Driver
import sys
sys.path.append(BASE_FOLDER)  # Ruta donde se encuentra la clase RecNBuilder y la funcion stratified_kfold

from naivebayes import RecNBBuilder, stratified_kfold, improved_clean_text, extract_sentiment_features, improved_tokenize, preprocess_to_tokens

In [77]:
# 2.4 ) Se ubica el archivo token generado por la cuenta Kaggle en Google drive
!mkdir -p ~/.kaggle
!cp "/content/drive/My Drive/Colab Notebooks/PROYECTO IA/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d kazanova/sentiment140
!unzip sentiment140.zip

# 2.5) La funcion importa los datos del archivo en CSV y lo guarda en unh DataFrame Pandas
df = pd.read_csv("training.1600000.processed.noemoticon.csv",
                 encoding="latin-1",
                 header=None)

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  sentiment140.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: training.1600000.processed.noemoticon.csv  


# 3. EXPLORACION DE DATOS

In [78]:
# 3.1) # Con este metodo de Data Frame Pandas se exploran los datos de las primeras cinco primer filas del
       #Data Set a partir de la posicion 0.
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [79]:
# 3.2 ) # Con este metodo se explora la informacion del DataFrame de pandas
        # como por ejemplo tipo de indice ,numero de filas y columnas, tipo de variables, el numero de valores No nulos ,uso de
        # memoria etc.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   0       1600000 non-null  int64 
 1   1       1600000 non-null  int64 
 2   2       1600000 non-null  object
 3   3       1600000 non-null  object
 4   4       1600000 non-null  object
 5   5       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


# 3.3) Informacion relevante obtenida de la exploracion del DataFrame
# Sentiment140:
#   0 => negative, 2 => neutral, 4 => positive
# columns: [target, id, date, flag, user, text]

In [80]:
# 3.3) Este metodo sirve para definir el nombre del encabezado de los atributos del DataFrame
df.columns = ["target","id","date","flag","user","text"]

# 3.4)  Con este metodo de Data Frame Pandas se exploran los datos de las primeras cinco primer filas del
       #Data Set a partir de la posicion 0.
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [81]:
# 3.5 )  Filtrar y balancear QUITAR los neutros (target==2)
df = df[df["target"].isin([0, 4])].copy()

# 3.6) Mapear a etiquetas palabra
df["sentiment"] = df["target"].map({0: "neg", 4: "pos"})

# 3.7) SUB‑MUESTREO para balancear las dos clases
min_size = df["sentiment"].value_counts().min()  # tamaño de la minoritaria
df = pd.concat([
        df[df["sentiment"] == "neg"].sample(min_size, random_state=0),
        df[df["sentiment"] == "pos"].sample(min_size, random_state=0)
]).sample(frac=1, random_state=0).reset_index(drop=True)


print(df["sentiment"].value_counts())


sentiment
neg    800000
pos    800000
Name: count, dtype: int64


4. LIMPIEZA DE DATOS
5. PROCESAMIENTO DE DATOS

In [82]:
# 4.1 ) Limpieza / Preprocesamiento

# Aplicar el preprocesamiento mejorado al dataset
print("\nAplicando limpieza y tokenización mejorada...")

# Crear nuevas columnas en el DataFrame
df['clean_text'] = df['text'].apply(improved_clean_text)
df['features'] = df['clean_text'].apply(extract_sentiment_features)
df['tokens'] = df.apply(lambda row: improved_tokenize(row['clean_text'], row['features']), axis=1)

# Mostrar algunos ejemplos del procesamiento
print("\nEjemplos de texto procesado:")
for i in range(5):
    print(f"\nTexto original: {df['text'].iloc[i]}")
    print(f"Texto limpio: {df['clean_text'].iloc[i]}")
    print(f"Tokens: {df['tokens'].iloc[i][:10]}..." if len(df['tokens'].iloc[i]) > 10 else f"Tokens: {df['tokens'].iloc[i]}")




Aplicando limpieza y tokenización mejorada...

Ejemplos de texto procesado:

Texto original: @el_gray Well, I'm off to Super Target to stimulate the economy then. Would have been my first via-Twitter transaction 
Texto limpio: USER well i m off to super target to stimulate the economy then would have been my first via twitter transaction
Tokens: ['USER', 'well', 'm', 'super', 'target', 'stimulate', 'economy', 'first', 'via', 'twitter']...

Texto original: @hellaeva OH BABY EVA! I'm sorry  wtf dicks. Thats insane, well honestly, i might be here next year so don't you worry (thanks Chem!)
Texto limpio: USER oh baby eva EXCL i m sorry wtf dicks thats insane well honestly i might be here next year so do not you worry thanks chem EXCL
Tokens: ['USER', 'oh', 'baby', 'eva', 'EXCL', 'm', 'sorry', 'wtf', 'dicks', 'thats']...

Texto original: Potong rambut 
Texto limpio: potong rambut
Tokens: ['potong', 'rambut', 'potong_rambut']

Texto original: is mad at myself cuz i didnt buy Lines Vines and

6. ENTRENAMIENTO Y EVALUACION DEL ALGORITMO DE APRENDIZAJE

In [83]:
#  Convertir columnas del DataFrame a arrays numpy para procesamiento eficiente
tokens_array = np.array(df["tokens"], dtype=object)  # Array de tokens/textos (features)
labels_array = np.array(df["sentiment"], dtype=object)  # Array de etiquetas/sentimientos (target)

#  Crear array con índices numéricos de todas las muestras (0 a N-1)
all_idx = np.arange(len(df))  # Ejemplo: [0, 1, 2, ..., 999] para 1000 muestras

#  Generar folds estratificados usando nuestra función personalizada

folds = stratified_kfold(y=df["sentiment"], k=90)  # 5 folds manteniendo proporción de clases

# 6.1) Evaluacion y Validacion del modelo de aprendizaje

# Lista para almacenar las precisiones de cada fold
accuracies = []


k=1 # Contador de iteraciones por cada Fold
# Iterar sobre cada fold (i = número de fold, test_idx = índices de test)
for i, test_idx in enumerate(folds, start=1):
    # Asegurar que los índices de test sean enteros para indexación válida
    test_idx = np.array(test_idx, dtype=int)

    # Obtener índices de entrenamiento como complemento de los de test
    train_idx = np.setdiff1d(all_idx, test_idx, assume_unique=True)  # Resta de conjuntos

    # Dividir datos usando los índices
    X_train = tokens_array[train_idx]  # Features de entrenamiento
    y_train = labels_array[train_idx]  # Labels de entrenamiento
    X_test  = tokens_array[test_idx]   # Features de test
    y_test  = labels_array[test_idx]   # Labels de test (ground truth)

# 6.2) Configurar y construir el modelo (Naive Bayes con suavizado de Laplace)
    builder = RecNBBuilder().set_laplace(2.5)  # Suavizado para evitar prob=0
    model   = builder.build()  # Instanciar modelo

# 6.3) Entrenar con los datos de entrenamiento del fold actual
    model.fit(X_train, y_train)  # Aprender patrones de los datos

# 6.4) Guardar el modelo en Google Drive sin comprimirlo ya que el archivo es menor a 100 MB
    dump(model, '/content/drive/My Drive/Colab Notebooks/PROYECTO IA/modelo_entrenado.joblib')

# 6.5) Calcular precisión en el conjunto de test del fold
    report = model.metrics_report(X_test, y_test)


# 6.6) Imprimir métricas por clase
    print("Iteracion", k, ":")
    for clase, valores in report['per_class'].items():
       print(f"Clase {clase}:")
       print(f"  Precisión: {valores['precision']:.3f}")
       print(f"  Recall: {valores['recall']:.3f}")
       print(f"  F1: {valores['f1']:.3f}")

    # Contador + 1 de iteraciones por cada Fold
    k=k+1

# 6.7) Imprimir macro promedios
print("\nMacro promedios:")
print(f"Precisión: {report['macro']['precision']:.3f}")
print(f"Recall: {report['macro']['recall']:.3f}")
print(f"F1: {report['macro']['f1']:.3f}")

Iteracion 1 :
Clase neg:
  Precisión: 0.794
  Recall: 0.808
  F1: 0.801
Clase pos:
  Precisión: 0.804
  Recall: 0.791
  F1: 0.798
Iteracion 2 :
Clase neg:
  Precisión: 0.794
  Recall: 0.803
  F1: 0.799
Clase pos:
  Precisión: 0.801
  Recall: 0.792
  F1: 0.796
Iteracion 3 :
Clase neg:
  Precisión: 0.805
  Recall: 0.808
  F1: 0.806
Clase pos:
  Precisión: 0.807
  Recall: 0.804
  F1: 0.805
Iteracion 4 :
Clase neg:
  Precisión: 0.794
  Recall: 0.796
  F1: 0.795
Clase pos:
  Precisión: 0.795
  Recall: 0.794
  F1: 0.794
Iteracion 5 :
Clase neg:
  Precisión: 0.799
  Recall: 0.801
  F1: 0.800
Clase pos:
  Precisión: 0.801
  Recall: 0.799
  F1: 0.800
Iteracion 6 :
Clase neg:
  Precisión: 0.791
  Recall: 0.800
  F1: 0.796
Clase pos:
  Precisión: 0.798
  Recall: 0.789
  F1: 0.793
Iteracion 7 :
Clase neg:
  Precisión: 0.794
  Recall: 0.807
  F1: 0.800
Clase pos:
  Precisión: 0.804
  Recall: 0.790
  F1: 0.797
Iteracion 8 :
Clase neg:
  Precisión: 0.801
  Recall: 0.805
  F1: 0.803
Clase pos:
  Preci

7. PREDICCION DE LA CLASE

In [1]:
# 7.1 )
# Permite cargar el  modelo entrenado de machine learning Naive Bayes en Google Drive
from joblib import load
# Asegurar que la clase esté disponible
from naivebayes import RecNaiveBayes  # Asegurar que la clase esté disponible

#7.1.1 ) # Cargar el modelo guardado en Google Drive
modelo_cargado = load('/content/drive/My Drive/Colab Notebooks/PROYECTO IA/modelo_entrenado.joblib')

#7.2 Comprobacion con tweet fuera de prueba
tweet_nuevo = "That moment when the movie buffer icon becomes the main character."
tokens_nuevo  = preprocess_to_tokens(tweet_nuevo)   # ← limpieza + tokenización
prediccion = modelo_cargado.predict([tokens_nuevo])
print("Tweet nuevo:", tweet_nuevo)
print("Predicción:", prediccion[0])

ModuleNotFoundError: No module named 'naivebayes'